In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.stats as stats

import utils
import matplotlib.pyplot as plt
import cvxpy as cp

import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm

In [2]:
from utils import *

In [3]:
nums = {'pinterest-20':9916, 'yelp':25815, 'ml-20m':20720}

datasets = ['ml-20m', 'pinterest-20', 'yelp']

models = ['EASE','MultiVAE','NeuMF','itemKNN','ALS']


metrics = ['Recall', 'NDCG', 'AP']

In [4]:
n = 500 # sample size + 1
gamma = 0.01
repeats = 100

In [5]:
for dataset in datasets:
    
    N = nums[dataset]
    
    F = np.diag(np.ones(N))
    g = np.zeros(N)

    A1 = np.ones(N).reshape(1, -1)
    b = 1.

    A = utils.A_Nn(N, n)
    AT = A.transpose()

    for model in models:

        print(model)


        # for fix sample
        save_path = '../save_PR/' + 'fix_sample_%d/'%n + dataset + '/MES/' + model + '/'

        if not os.path.exists(save_path):
                os.makedirs(save_path)

        for re in tqdm(range(repeats)):

            ru, Ru = fix_load_model(model, dataset, n, re)

            # weight
            xu = ru.copy()

            ru = ru + 1
            Ru = Ru + 1

            pr = get_pr(xu, n)


            x = cp.Variable(N)
            #obj = cp.Minimize(cp.sum(cp.multiply(cp.log(AT@x + c), pr) - gamma * cp.sum(cp.entr(x))))
            #obj = cp.Minimize(-cp.sum( cp.multiply(cp.log(AT + 1e-9)@x, pr)) - gamma * cp.sum(cp.entr(x)))
            obj = cp.Minimize(cp.sum(cp.square(AT@x - pr)@pr*n)- gamma * cp.sum(cp.entr(x)))
            constraints = [A1@x == b,
                           F@x >= g]

            prob = cp.Problem(obj, constraints)

            prob.solve(solver=cp.MOSEK, verbose=False)

            pi = np.array(x.value)

            np.savez(save_path + '%d.npz'%re, R = pi)

  0%|          | 0/100 [00:00<?, ?it/s]

EASE


  0%|          | 0/100 [00:00<?, ?it/s]

MultiVAE


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

itemKNN


  0%|          | 0/100 [00:00<?, ?it/s]

ALS


100%|██████████| 100/100 [39:01<00:00, 23.41s/it]
